# Seminar - Example of getting chain data
Getting current state of Carmine Options AMM from StarkNet.

In [ ]:
from dataclasses import dataclass
import asyncio

from starknet_py.net.full_node_client import FullNodeClient
from starknet_py.hash.selector import get_selector_from_name
from starknet_py.net.client_models import Call

In [ ]:
AMM_ADDRESS = 0x047472e6755afc57ada9550b6a3ac93129cc4b5f98f51c73e0644d129fd208d9 # Carmine Options Amm address

CUBIT_SHIFT = 2**64

# RPC setup
RPC_URL = 'https://starknet-mainnet.public.blastapi.io'
NET = FullNodeClient(RPC_URL)

In [ ]:
def felt_to_str(felt) -> str:
    """
    Helper function that takes felt and creates string out of it.
    """
    num_bytes = (felt.bit_length() + 7) // 8
    bytes_ = felt.to_bytes(num_bytes, "big")
    return bytes_.decode("utf-8")

In [ ]:
async def function_call(client: FullNodeClient, address: int, selector: str, calldata: list[int | None]) -> list[int]:
    """
    Wrapper for StarkNet chain calls. -> gets chain data
    """
    call = Call(
        to_addr=address,
        selector=get_selector_from_name(selector),
        calldata=calldata
    )
    res = await client.call_contract(call)
    return res

async def function_call_from_block(client: FullNodeClient, address: int, selector: str, calldata: list[int], block_num: int) -> list[int]:
    """
    Wrapper for StarkNet chain calls for HISTORICAL data. -> gets HISTORICAL chain data
    """
    call = Call(
        to_addr=address,
        selector=get_selector_from_name(selector),
        calldata=calldata
    )
    res = await client.call_contract(call, block_number=block_num)
    return res

In [ ]:
# Fetch list of lptokens
lptokens = await function_call(NET, AMM_ADDRESS, 'get_all_lptoken_addresses', [])
lptokens = lptokens[1:] # First element is length of the array 

In [ ]:
@dataclass
class Token:
    """
    Class representing single Starknet token.
    """
    decimals: int
    symbol: str
    address: int

    @classmethod
    async def from_address(cls, address: int, client: FullNodeClient) -> 'Token':
        decimals = await function_call(client, address, 'decimals', [])
        symbol = await function_call(client, address, 'symbol', [])

        return Token(
            decimals=decimals[0],
            symbol= felt_to_str(symbol[0]),
            address = address
        )

    def __repr__(self) -> str:
        return f"""Token(
    decimals={self.decimals},
    symbol={self.symbol},
    address={hex(self.address)}
)"""


@dataclass
class PoolInfo:
    """
    Class representing Carmine Options AMM pool info.

    Footnote: ETH/USDC, ETH is the base token and USDC is the quote token.
    """
    quote_token_address: int
    base_token_address: int
    option_type: int
    address: int

    @classmethod
    async def from_address(cls, address: int, client: FullNodeClient) -> 'PoolInfo':
        """
        Constructs Pool info from the address of the lptoken.
        """
        res = await function_call(
            client,
            AMM_ADDRESS, 
            'get_pool_definition_from_lptoken_address', [address]
        )
        return PoolInfo(
            quote_token_address=res[0],
            base_token_address=res[1],
            option_type=res[2],
            address=address
        )

    def __repr__(self) -> str:
        return f"""PoolInfo(
    quote_token_address={hex(self.quote_token_address)},
    base_token_address={hex(self.base_token_address)},
    option_type={self.option_type},
    address={hex(self.address)}
)"""

    

In [ ]:
tokens = [ Token.from_address(i, NET) for i in lptokens ]
tokens = await asyncio.gather(*tokens)

In [ ]:
tokens

In [ ]:
pool_infos = [ PoolInfo.from_address(i, NET) for i in lptokens ]
pool_infos = await asyncio.gather(*pool_infos)

In [ ]:
pool_infos

In [ ]:
for pool in pool_infos:
    # Fetch unlocked and value of position
    unlocked = await function_call(NET, AMM_ADDRESS, 'get_unlocked_capital', [pool.address])

    value_of_position = await function_call(NET, AMM_ADDRESS, 'get_value_of_pool_position', [pool.address])
    value_of_position = value_of_position[0] / CUBIT_SHIFT 

    base_token = await Token.from_address(pool.base_token_address, NET)
    quote_token = await Token.from_address(pool.quote_token_address, NET)

    if pool.option_type == 0: # Call options
        type_str = 'Call'
        capital_token = base_token.symbol

        pool_tvl = (unlocked[0] / 10**base_token.decimals) + value_of_position 
    else: # Put
        type_str = 'Put'
        capital_token = quote_token.symbol

        pool_tvl = (unlocked[0] / 10**quote_token.decimals) + value_of_position 


    print(
        '{:<17}'.format(f'TVL of {base_token.symbol}/{quote_token.symbol} ')
        + '{:<12}'.format(f'{type_str} pool: ')
        + '{:>9}'.format(str(float('%0.2f' % pool_tvl)))
        + f' {capital_token: >4}'
    )
    